In [3]:
import pandas as pd

### Download the data and load it to Pandas. 

You can find the data files in the README.

In [4]:
titles = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Python Practices/Pandas/Pandas_exercise/data/imdb_pandas/titles.csv')
titles.head()

,title,year
0,The Rising Son,1990
1,The Thousand Plane Raid,1969
2,Crucea de piatra,1993
3,Country,2000
4,Gaiking II,2011


In [5]:
cast = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Python Practices/Pandas/Pandas_exercise/data/imdb_pandas/cast.csv')
cast.head()

,title,year,name,type,character,n
0,Closet Monster,2015,Buffy #1,actor,Buffy 4,NaN
1,Suuri illusioni,1985,Homo $,actor,Guests,22.0
2,Battle of the Sexes,2017,$hutter,actor,Bobby Riggs Fan,10.0
3,Secret in Their Eyes,2015,$hutter,actor,2002 Dodger Fan,NaN
4,Steve Jobs,2015,$hutter,actor,1988 Opera House Patron,NaN


## Part 1: Group By

### 1. Using groupby(), count the number of films that have been released in each decade in the history of cinema.

In [6]:
titles['decade'] = titles['year'] // 10 * 10

titles

,title,year,decade
0,The Rising Son,1990,1990
1,The Thousand Plane Raid,1969,1960
2,Crucea de piatra,1993,1990
3,Country,2000,2000
4,Gaiking II,2011,2010
...,...,...,...
232325,E yue,2005,2000
232326,Lakota Girls,2015,2010
232327,Kroniki domowe,1997,1990
232328,Awakening the Zodiac,2017,2010


In [7]:
titles.groupby('decade').size()

decade
1890        3
1900       37
1910     6512
1920     8797
1930    10097
1940     8576
1950    12711
1960    17515
1970    18714
1980    20243
1990    22225
2000    38713
2010    67977
2020      210
dtype: int64

### 2. Use groupby() count the number of "Hamlet" films made in each decade.

In [8]:
titles[ titles['title'] == 'Hamlet'].groupby('decade').size()

decade
1910    3
1920    1
1940    1
1950    1
1960    2
1970    2
1980    1
1990    2
2000    2
2010    3
dtype: int64

### 3. Use groupby() to determine how many roles are listed for each of the Pink Panther movies.

In [9]:
(cast [ cast['title'].str.contains('Pink Panther', na=False)]
    .groupby('title')
    .agg( {
        'name' : 'count'
    })
    .sort_values(by='name',ascending=False)
 
)

,name
title,
The Pink Panther,111
The Pink Panther 2,84
The Pink Panther Strikes Again,68
Curse of the Pink Panther,67
Revenge of the Pink Panther,64
Son of the Pink Panther,46
Trail of the Pink Panther,41
The Return of the Pink Panther,30


In [10]:
# consideration for case sensitive titles

(cast [ cast['title'].str.contains('pink panther', case=False ,na=False)]
    .groupby('title')
    .agg( {
        'name' : 'count'
    })
    .sort_values(by='name',ascending=False)
 
)

,name
title,
The Pink Panther,111
The Pink Panther 2,84
The Pink Panther Strikes Again,68
Curse of the Pink Panther,67
Revenge of the Pink Panther,64
Son of the Pink Panther,46
Trail of the Pink Panther,41
The Return of the Pink Panther,30


### 4. Using groupby(), list each of the films in which Frank Oz has played more than 1 role in order by year.

In [11]:
# needed assistance to filter by >= 2 roles. Pandas has a QUERY METHOD

cast [ cast['name'] == 'Frank Oz']\
    .groupby(['title','year'])\
        .agg ({
            'name' : 'count'
            })\
             .query('name >= 2')\
                .sort_values(by='year')


,,name
title,year,
The Muppet Movie,1979,8
An American Werewolf in London,1981,2
The Great Muppet Caper,1981,6
The Dark Crystal,1982,2
The Muppets Take Manhattan,1984,7
Follow That Bird,1985,3
The Muppet Christmas Carol,1992,7
Muppet Treasure Island,1996,4
Muppets from Space,1999,4


## Part 2: Merging

For these questions, you'll also need the following dataframe on release dates.

You'll need to merge to join together the information present in different dataframes, in addition to other filtering and calculations.

In [12]:
release_dates = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Python Practices/Pandas/Pandas_exercise/data/imdb_pandas/release_dates.csv', index_col=None,
                                      parse_dates=['date'], infer_datetime_format=True)
release_dates.head()

/var/folders/bs/_y1b6rb96rb21tm4p3r5fpz80000gn/T/ipykernel_72394/3958197310.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  release_dates = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Python Practices/Pandas/Pandas_exercise/data/imdb_pandas/release_dates.csv', index_col=None,


,title,year,country,date
0,"#73, Shaanthi Nivaasa",2007,India,2007-06-15
1,#Beings,2015,Romania,2015-01-29
2,#Declimax,2018,Netherlands,2018-01-21
3,#Ewankosau saranghaeyo,2015,Philippines,2015-01-21
4,#Horror,2015,USA,2015-11-20


### 1. Count the number of movies with "Christmas" in their title for each month, that is released are the USA.

In [13]:
(
    release_dates[ 
        (release_dates['title'].str.contains('Christmas', case=False, na=False)) & 
        (release_dates['country'] == 'USA')]
        .groupby(release_dates['date'].dt.month)
        .size()
        
)

date
1      2
2      2
4      1
5      1
6      1
7      1
8      2
9      2
10    14
11    38
12    47
dtype: int64

### 2. Count the number of movies with "Romance" in their title for **each day of the week**, that are released in the USA.

In [14]:
(
    release_dates [ (release_dates['title'].str.contains('Romance')) &
                   (release_dates['country'] == 'USA')]
                   .groupby(release_dates['date'].dt.day_of_week)
                   .size()
)

date
0    21
1     4
2    11
3     6
4    18
5     8
6    32
dtype: int64

In [15]:
# recommended improvement on clarity by importing calender

import calendar

(
    release_dates [ (release_dates['title'].str.contains('Romance')) &
                   (release_dates['country'] == 'USA')]
                   .groupby(release_dates['date'].dt.day_of_week)
                   .size()
                    .rename(index=dict(enumerate(calendar.day_name)))
)

date
Monday       21
Tuesday       4
Wednesday    11
Thursday      6
Friday       18
Saturday      8
Sunday       32
dtype: int64

### 3. On which date was each Judi Dench movie from the 1990s released in the USA?

In [20]:
# Used assistance to confirm how to merge on TWO criteria.

cast['decade'] = cast['year'] // 10 * 10

cast_release = pd.merge(cast,release_dates, on=['title','year'], how='inner')

(
    cast_release[ (cast_release['name'] == 'Judi Dench') &
                 (cast_release['country'] == 'USA') &
                 (cast_release['decade'] == 1990)]
                 [['title','name','year','date']]
               
)

,title,name,year,date
14166354,GoldenEye,Judi Dench,1995,1995-11-17
14166393,Hamlet,Judi Dench,1996,1996-12-25
14166551,Jack & Sarah,Judi Dench,1995,1996-03-22
14166668,Mrs Brown,Judi Dench,1997,1997-10-03
14167083,Shakespeare in Love,Judi Dench,1998,1999-01-08
14167283,Tea with Mussolini,Judi Dench,1999,1999-05-14
14167509,The World Is Not Enough,Judi Dench,1999,1999-11-19
14167582,Tomorrow Never Dies,Judi Dench,1997,1997-12-19


In [66]:
cast.head(5).sort_values(by='title')

,title,year,name,type,character,n
2,Battle of the Sexes,2017,$hutter,actor,Bobby Riggs Fan,10.0
0,Closet Monster,2015,Buffy #1,actor,Buffy 4,NaN
3,Secret in Their Eyes,2015,$hutter,actor,2002 Dodger Fan,NaN
4,Steve Jobs,2015,$hutter,actor,1988 Opera House Patron,NaN
1,Suuri illusioni,1985,Homo $,actor,Guests,22.0


In [68]:
release_dates.head(5).sort_values(by='title')

,title,year,country,date
0,"#73, Shaanthi Nivaasa",2007,India,2007-06-15
1,#Beings,2015,Romania,2015-01-29
2,#Declimax,2018,Netherlands,2018-01-21
3,#Ewankosau saranghaeyo,2015,Philippines,2015-01-21
4,#Horror,2015,USA,2015-11-20


### 4. In which months do films with the actress Judi Dench tend to be released in the USA?

In [22]:
import calendar
(
    cast_release [ (cast_release['name'] == 'Judi Dench') &
                  (cast_release['country'] == 'USA')]
                  .groupby(cast_release['date'].dt.month)
                  .size()
                  .rename(index=dict(enumerate(calendar.month_name)))
)

date
January       4
February      4
March         3
April         2
May           4
June          3
July          1
August        1
September     2
October       1
November     12
December      4
dtype: int64